# Lab 1-1 "Infrastructure"

## Lab 1-1 description

## Environment

In [1]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 24.04.4 LTS
Release:	24.04
Codename:	noble


In [2]:
!python --version

Python 3.12.3


In [3]:
!pip freeze | grep -E "langchain|transformers"

langchain==1.2.10
langchain-chroma==1.1.0
langchain-classic==1.0.1
langchain-community==0.4.1
langchain-core==1.2.15
langchain-huggingface==1.2.0
langchain-ollama==1.0.1
langchain-text-splitters==1.1.1
sentence-transformers==5.2.2
transformers==4.57.6


In [4]:
!ollama --version

]11;?\ollama version is 0.17.0


In [5]:
!ollama list

]11;?\NAME             ID              SIZE      MODIFIED       
qwen3:4b         359d7dd4bcda    2.5 GB    24 minutes ago    
llama3:latest    365c0bd3c000    4.7 GB    31 minutes ago    
phi3:latest      4f2222927938    2.2 GB    21 hours ago      
qwen:0.5b        b5dc5e784f2a    394 MB    22 hours ago      


# Task 1: simple LLM response

In [6]:
from langchain_ollama import OllamaLLM  

llm = OllamaLLM(model="qwen:0.5b", temperature=0.7)
response = llm.invoke("what is langchain?")
print("Ответ модели:\n", response)

/home/ychernyshov/urfu/1/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ответ модели:
 Langchain is a programming language that was created by Google. Langchain's syntax is similar to other modern programming languages, such as Python and Java.

One of the key features of Langchain is its ability to execute arbitrary code. This allows developers to write modular, reusable code that can be easily integrated into other applications or systems.

Another key feature of Langchain is its ability to handle exceptions in execution. This allows developers to design robust and error-free programs that are resistant to unexpected errors.

Overall, Langchain's syntax, ability to execute arbitrary code, and handling exceptions make it an attractive choice for developers looking to write modular, reusable code that can be easily integrated into other applications or systems.


In [7]:
llm = OllamaLLM(model="qwen3:4b", temperature=0)
response = llm.invoke("what is langchain?")
print("Ответ модели:\n", response)

Ответ модели:
 **LangChain** is an **open-source framework** designed to help developers build **applications powered by large language models (LLMs)**. It simplifies the process of integrating LLMs (like OpenAI's GPT, Anthropic's Claude, or Meta's Llama) into real-world applications by providing tools for:

1. **Connecting LLMs to data** (e.g., databases, APIs, files)
2. **Creating chains of reasoning** (e.g., step-by-step workflows)
3. **Handling context** (e.g., remembering past interactions)
4. **Building agents** (e.g., chatbots that can search the web or execute tasks)
5. **Evaluating and improving** LLM outputs

### Why is LangChain important?
- **Solves a real problem**: Building apps with LLMs is complex (e.g., managing context, handling errors, connecting to external data). LangChain abstracts these challenges.
- **Modular & flexible**: Developers can plug in different tools (e.g., vector databases, APIs) without rewriting everything.
- **Community-driven**: It’s widely used 

# Task 2: prompt chain

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain
from langchain_ollama import OllamaLLM
import time

In [9]:
prompt = ChatPromptTemplate.from_template(
    "Ты — эксперт по {тема}. Объясни понятие '{понятие}' кратко и понятно."
)
llm = OllamaLLM(model="qwen:0.5b")
chain = prompt | llm  # синтаксическая цепочка (LangChain 0.2+)

start = time.time()
result = chain.invoke({"тема": "LLM security", "понятие": "jailbrake"})
print(result, time.time() - start)

'JailBrake' означает, что следующий код будет использоваться в качестве аргумента к производительности. 

Вот пример кода:

```
public class MyClass {
    public static void main(String[] args) {
        System.out.println("Hello World");
    }
}
```

В этом примере кода будет использоваться в качестве аргумента к производительности, поскольку код мышечного и функционального, который будет использоваться при выполнении различных задач. 8.79245924949646


In [10]:
prompt = ChatPromptTemplate.from_template(
    "Ты — эксперт по {тема}. Объясни понятие '{понятие}' кратко и понятно."
)
llm = OllamaLLM(model="qwen3:4b")
chain = prompt | llm  

start = time.time()
result = chain.invoke({"тема": "LLM security", "понятие": "jailbrake"})
print(result, time.time() - start)

**Jailbreak** — это техника, которую злоумышленник использует для обхода внутренних ограничений безопасности LLM (например, правил, запрещающих дискриминацию или утечку конфиденциальной информации) и заставляет модель генерировать нежелательный или запрещенный контент. Пример: заставить модель обсуждать военные действия, если это было запрещено в её обучении. 1219.086008310318


# Task 3: RAG

In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
from langchain_ollama import OllamaLLM

In [12]:
# 1. Загрузка документа (создайте файл data.txt с текстом)
loader = TextLoader("data.txt", encoding="utf-8")
docs = loader.load()

# 2. Разделение на чанки
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# 3. Векторное хранилище
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embeddings)

# 4. Цепочка QA
llm = OllamaLLM(model="phi3")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# 5. Запрос
query = "О чём основной текст?"
result = qa_chain.invoke(query)
print("Ответ:", result["result"])
print("Источник:", result["source_documents"][0].page_content[:100] + "...")

Ответ: The main text is from Lewis Carroll's poem "Jabberwocky," which appears in his novel "Through the Looking-Glass, and What Alice Found There." The excerpt provided includes nonsensical words used throughout the entirety of this famous work.
Источник: And, as in uffish thought he stood,
The Jabberwock, with eyes of flame,
Came whiffling through the t...


# Task 4: Agents

In [13]:
from langchain_classic.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain_core.tools import Tool, tool
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_core.prompts import PromptTemplate

In [14]:
@tool
def calculate(expression: str) -> str:
    """
    Вычисляет математическое выражение.
    Поддерживает: +, -, *, /, **, скобки.
    Пример: "2 + 2 * 3", "(10 - 5) * 4", "2 ** 10"
    Используйте этот инструмент для любых математических расчётов.
    """
    try:
        # Очищаем выражение от опасных символов (безопасность!)
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Ошибка: Недопустимые символы в выражении"
        
        # Вычисляем результат
        result = eval(expression)
        return f"Результат: {result}"
    
    except ZeroDivisionError:
        return "Ошибка: Деление на ноль"
    except SyntaxError:
        return "Ошибка: Неверный синтаксис выражения"
    except Exception as e:
        return f"Ошибка: {str(e)}"

# Список доступных инструментов
tools = [calculate]

In [15]:
llm = ChatOllama(model="qwen3:4b", temperature=0)

In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты полезный ассистент с доступом к калькулятору.
    
ПРАВИЛА:
1. Для любых математических вычислений используй инструмент 'calculate'
2. Не пытайся вычислять в уме сложные выражения
3. Если пользователь задаёт вопрос с числами — используй калькулятор
4. Отвечай на русском языке"""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [17]:
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,           # Показывать ход рассуждений
    max_iterations=5,       # Максимум попыток
    handle_parsing_errors=True,  # Обрабатывать ошибки парсинга
)

In [18]:
while True:
        try:
            user_input = input("\nВы: ")
            if user_input.lower() in ["exit", "quit", "выход"]:
                print("До свидания!")
                break
            
            response = agent_executor.invoke({"input": user_input})
            print(f"Агент: {response['output']}")
            
        except KeyboardInterrupt:
            print("\nДо свидания!")
            break
        except Exception as e:
            print(f"Ошибка: {e}")


Вы:  2+2




> Entering new AgentExecutor chain...

Invoking: `calculate` with `{'expression': '2 + 2'}`


Результат: 4Результат вычисления: 4

> Finished chain.
Агент: Результат вычисления: 4



Вы:  exit


До свидания!


# References